# Heel simpel ML-voorbeeld
Voorbeelden voor verbeterputen in het boek: http://i.amcat.nl/ccsbook_preview/chapter11.html#11_4

In [20]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import metrics

## 1. Data inlezen en opschonen
We hebben in principe twee kolommen in een dataframe (of twee losse lijsten, dat kan ook) nodig: eentje met texten, eentje met labels. We doen het nu met twee labels, maar het kan net zo goed met drie of vier, dan gaat sckikit-learn automatisch meerdere modellen maken - het is voor jullie precies dezelfde workflow.

In [12]:
df = pd.read_excel("Alle_4599_chunks_EN_news_metchunks_250_Alle3bijelkaar.xlsx")
data = df[[3,'sentiment Lisa']]
data.columns=['text','label']
data = data.dropna()
data

,text,label
0,Immunetherapy Scientists are now trying to ta...,neu
1,The immune-therapy is based on a biolog...,neu
2,Then there are immune therapies and blood tr...,neu
3,14 Length: 125 words Byline: MARILYNN MARCHI...,pos
4,Hopes for immuno-therapies are now so hyped-u...,pos
...,...,...
245,The team also saw an increase in the fluoresce...,neu
246,Danielle passed away when twins Colby and Re...,neu
247,Tom will move on to an intensive course of im...,neu
248,Tom will move on to an intensive course of im...,neu


In [13]:
data['label'].value_counts()

pos         79
neu         56
neg          6
neu?         5
pos?         1
both         1
neu/pos?     1
neg?         1
Name: label, dtype: int64

In [15]:
def recode(s):
    if s[:3]=='pos':
        return 'pos'
    elif s[:3]=='neu':
        return 'neu'
    else:
        return None

In [17]:
data['label'] = data['label'].apply(recode)
data = data.dropna()
data

,text,label
0,Immunetherapy Scientists are now trying to ta...,neu
1,The immune-therapy is based on a biolog...,neu
2,Then there are immune therapies and blood tr...,neu
3,14 Length: 125 words Byline: MARILYNN MARCHI...,pos
4,Hopes for immuno-therapies are now so hyped-u...,pos
...,...,...
245,The team also saw an increase in the fluoresce...,neu
246,Danielle passed away when twins Colby and Re...,neu
247,Tom will move on to an intensive course of im...,neu
248,Tom will move on to an intensive course of im...,neu


## 2. Train-test-split
Ervan uitgaande dat de zinnen random zijn (dat is wel belangrijk, anders eerst shuffelen) maken we een training en en een test dataset aan, ongeveer verouding 70:30 of zo. We pakken dus de eerste honderd en de laatste 42

In [18]:
train = data[:100]
test = data[100:]

In [19]:
len(train), len(test)

(100, 42)

# 3. Vectorizer en model fitten en schatten

- fitten van vectorizer: bepalen welke kolommen (woorden) we willen hebben
- transformeren: daadwerkelijk de woorden per artikel tellen, dus de kolommen vullen zodat we een AANTAL_ARTIKELEN * AANTAL_FEATURES(=woorden) matrix hebben
- heeeeeel erg belangrijk: de vectorizer later (voor testdataset of voor nieuwe "unseen" data voor voorspellingen *niet* opnieuw fitten, anders komen de kolommen niet meer overeen

In [21]:
myvec = CountVectorizer()

In [23]:
X_train = myvec.fit_transform(train['text'])
X_test = myvec.transform(test['text'])

In [25]:
X_train

<100x932 sparse matrix of type '<class 'numpy.int64'>'
	with 2717 stored elements in Compressed Sparse Row format>

In [27]:
nb = MultinomialNB()
nb.fit(X_train, train["label"])

MultinomialNB()

## 4. Vergelijken met testdataset

Voor de testdata weten we de daadwerkelijke labels, maar ons model kent ze niet. Dus kunnen we gewoon kijken of de voorspellingen van het model overeenkomen met de daadwerkelijke labels

In [29]:
labels_predicted = nb.predict(X_test)

In [32]:
print(metrics.classification_report(test['label'], labels_predicted))

              precision    recall  f1-score   support

         neu       0.56      0.26      0.36        19
         pos       0.58      0.83      0.68        23

    accuracy                           0.57        42
   macro avg       0.57      0.54      0.52        42
weighted avg       0.57      0.57      0.53        42



## 5. Classifier gebruiken
Ervan uitgaande dat we hiermee tevreden zijn (eigenlijk zouden we dus eerst nog meerdere modellen vergelijken etc, zie boek), kunnen we nu de classifier gebruiken om het sentiment van milioenen andere zinnen te voorspellen (dat is natuurlijk wat we willen.

In [45]:
# kun je natuurlijk beter direct uit een bestand of een dataframe of wat dan ook pakken, 
# ik maak even een lijst met voorbeeldzinnen omdat ik te lui ben ;-)
nieuwedata = ["This is a sentence Damian made up and which is about immuneotherapie which is a great thing!", 
             "Another sentence eh rkjhrkj hrt h",
             "And a thid really shitty sentence"]

In [46]:
X = myvec.transform(nieuwedata)
predicted_labels=nb.predict(X)

In [47]:
pd.DataFrame({"sentence":nieuwedata, "label":predicted_labels})

,sentence,label
0,This is a sentence Damian made up and which is...,neu
1,Another sentence eh rkjhrkj hrt h,pos
2,And a thid really shitty sentence,pos
